In [2]:
from pathlib import Path
import numpy as np
from cellpose import io, models
import matplotlib.pyplot as plt



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.4 
platform:       	win32 
python version: 	3.13.7 
torch version:  	2.7.1+cpu! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




In [ ]:

IMG_DIR    = Path(r"C:\Users\mudzimtb\Downloads\Spheroids\Day1\A")
PATTERN    = "*.tif"
MODEL_PATH = r"C:\Users\mudzimtb\Downloads\Spheroids\Day1\New folder\models\cpsam_20250904_165506"

DIAMETER   = None      # or e.g. 75.0
USE_GPU    = False


def save_preview(img, masks, out_path):
    """
    Save a visualization of masks over the original image.
    Grey background + colored masks overlay.
    """
    plt.figure(figsize=(6,6))
    # show raw image as background (grey)
    if img.ndim == 2:   # grayscale
        plt.imshow(img, cmap="gray")
    elif img.ndim == 3 and img.shape[-1] in (3,4):  # RGB
        plt.imshow(img)
    else:  # if stack, show first plane
        plt.imshow(img[0], cmap="gray")

    # overlay mask with transparency
    plt.imshow(masks, alpha=0.4, cmap="nipy_spectral")
    plt.axis("off")
    plt.savefig(out_path, dpi=200, bbox_inches='tight', pad_inches=0)
    plt.close()

def run_cellpose_v4(model, img, diameter):
    if img.ndim == 2:  # grayscale 2D
        masks, *_ = model.eval(img, diameter=diameter, channels=None, channel_axis=None)
        return masks.astype(np.uint16)
    if img.ndim == 3 and img.shape[-1] in (3,4):  # RGB
        masks, *_ = model.eval(img, diameter=diameter, channels=None, channel_axis=-1)
        return masks.astype(np.uint16)
    if img.ndim == 3:  # stack (T,H,W)
        planes = []
        for k in range(img.shape[0]):
            masks, *_ = model.eval(img[k], diameter=diameter, channels=None, channel_axis=None)
            planes.append(masks.astype(np.uint16))
        return np.stack(planes, axis=0)
    raise ValueError(f"Unsupported shape {img.shape}")

def main():
    model = models.CellposeModel(pretrained_model=MODEL_PATH, gpu=USE_GPU)
    for img_path in sorted(IMG_DIR.glob(PATTERN)):
        print(f"Processing {img_path.name}...")
        img = io.imread(str(img_path))
        masks = run_cellpose_v4(model, img, DIAMETER)

        # save raw mask
        mask_out = img_path.with_name(img_path.stem + "_masks.tif")
        io.imsave(str(mask_out), masks)

        # save preview overlay
        preview_out = img_path.with_name(img_path.stem + "_masks_preview.png")
        save_preview(img, masks, preview_out)

        print(f"  Saved mask: {mask_out}")
        print(f"  Saved preview: {preview_out}")

if __name__ == "__main__":
    main()


Processing A01_x0_y0_w0.tif...
  Saved mask: C:\Users\mudzimtb\Downloads\Spheroids\Day1\A\A01_x0_y0_w0_masks.tif
  Saved preview: C:\Users\mudzimtb\Downloads\Spheroids\Day1\A\A01_x0_y0_w0_masks_preview.png
Processing A02_x0_y0_w0.tif...
  Saved mask: C:\Users\mudzimtb\Downloads\Spheroids\Day1\A\A02_x0_y0_w0_masks.tif
  Saved preview: C:\Users\mudzimtb\Downloads\Spheroids\Day1\A\A02_x0_y0_w0_masks_preview.png
Processing A03_x0_y0_w0.tif...
  Saved mask: C:\Users\mudzimtb\Downloads\Spheroids\Day1\A\A03_x0_y0_w0_masks.tif
  Saved preview: C:\Users\mudzimtb\Downloads\Spheroids\Day1\A\A03_x0_y0_w0_masks_preview.png
Processing A04_x0_y0_w0.tif...
  Saved mask: C:\Users\mudzimtb\Downloads\Spheroids\Day1\A\A04_x0_y0_w0_masks.tif
  Saved preview: C:\Users\mudzimtb\Downloads\Spheroids\Day1\A\A04_x0_y0_w0_masks_preview.png
Processing A09_x0_y0_w0.tif...
  Saved mask: C:\Users\mudzimtb\Downloads\Spheroids\Day1\A\A09_x0_y0_w0_masks.tif
  Saved preview: C:\Users\mudzimtb\Downloads\Spheroids\Day1\A\A